# Calculate Nino 3.4 index to check if eruptions line up with El Nino events
- opens all the C20C (20th century reanalysis data) files with data on precipitation prate and surface temperature tmax
- calculates the Nino3.4 index for the data to test the ENSO state in the months before and after Krakatoa 1883

In [1]:
import xarray as xr, matplotlib.pyplot as plt
from importlib import reload # need to use this if I edit a function file
import os
import numpy as np
import pandas as pd
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????
import dask.diagnostics # dask allows you to check how long something is taking to load
import climtas # needed to count event statistics with a specified duration

In [2]:
# import custom functions
import sys 
sys.path.append('/home/563/kb6999/Functions') # use this if the function file is in a different directory to the notebook

import frequently_used_functions as func
import plotting_functions as fplot
import model_functions as funcM
import reanalysis_functions as funcR

# problem with chunks when calcultating nino34 index

In [3]:
# read in precipitation data
ds_prate = xr.open_mfdataset('/g/data/ua8/C20C/v3/mean_daily/prate/*.nc', combine='by_coords')#, chunks={'time': 12})

In [4]:
# read in temperature data
ds_tmax = xr.open_mfdataset('/g/data/ua8/C20C/v3/mean_daily/tmax_2m/*.nc', combine='by_coords')#, chunks={'time': 12})

In [5]:
# combine precipitation and temperature into one dataset
C20C = xr.Dataset({'tmax': ds_tmax.tmax, 'prate': ds_prate.prate})
# change the pr units
C20C['prate'] = C20C.prate*86400

In [6]:
# select out the same time period as the models
C20C = C20C.sel(time=slice('1850-01', '2015-01'))

## Area weighting and land frac

In [7]:
# import landmask dataset 
landfrac_ds = xr.open_dataset('/g/data/ua8/C20C/v3/LAND.nc')
landmask = landfrac_ds.mean(dim='time')

In [8]:
# area weighting 
C20C_w = C20C*np.cos(C20C.lat*(np.pi/180))

## Group data by month

In [9]:
# group the data by month (take the monthly mean)
C20C_mon = C20C_w.resample(time='M').mean(dim='time')

In [10]:
C20C_mon

<xarray.Dataset>
Dimensions:  (lat: 181, lon: 360, time: 1981)
Coordinates:
  * time     (time) datetime64[ns] 1850-01-31 1850-02-28 ... 2015-01-31
  * lat      (lat) float32 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
Data variables:
    tmax     (time, lat, lon) float32 dask.array<chunksize=(1, 181, 360), meta=np.ndarray>
    prate    (time, lat, lon) float64 dask.array<chunksize=(1, 181, 360), meta=np.ndarray>

## Nino 3.4

In [ ]:
# calculate the nino3.4 index for every month divided by std
nino34_index = func.nino34(C20C_mon.load(), '1850-01', '1880-01', 1)

In [ ]:
# set the threshold for el nino/la nia events
THOLD = 0.4

## Graphs

In [53]:
# eruption date
e_date = pd.to_datetime(['1883-08','1902-10', '1963-03', '1982-04', '1991-06'])

In [ ]:
# plot time series of nino3.4 index for the multimodel mean dataset 
fig, axes = plt.subplots(figsize=(16,6))

fplot.nino34_plot(nino34_mmm.tmax.sel(time=slice('1880-01','1889-12')), e_date, THOLD, ax = axes)

plt.savefig('R_nino34_plot_20C.png', dpi=300, bbox_inches='tight')

## Table of ENSO events
- count the number of el nino/la nina/ neutral events each year 
- record count of dominant event for the year
- write the count and colour code (red el nino, blue la nina, white/grey neutral)  

In [ ]:
# select out the years near the eruption
nino34_index = nino34_index.sel(time=slice('1882-01','1887-12'))

In [ ]:
# find the nino/nina events over the specified threshold and lasting longer than 5 months
ENSO_events_04 = find_ENSO(nino34_index, 0.4)